In [2]:
!pip install ta
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dropout,GRU
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

df = pd.read_csv("./[경기관련소비재]현대차.csv")
df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.columns = df_columns
df = df.sort_values('Date')
df['Date']= df['Date'].astype('str')
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
    df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

df.dropna(inplace=True)


import ta

H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

scaler1 = MinMaxScaler()

temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

scaled_df.insert(3, 'Close',temp)
scaled_df

df = scaled_df



import numpy as np

def make_dataset(feature, label, window_size = 20):
  
  feature_list = []
  label_list = []
  
  for i in range(len(feature) - window_size) :
    feature_list.append(feature.iloc[i:i+window_size])
    label_list.append(label.iloc[i+window_size])
  
  return np.array(feature_list), np.array(label_list)

WINDOW_SIZE=20

result = []
predict = []


####최근 20개의 데이터만
pred_feature = df[-WINDOW_SIZE :]
pred_feature = pred_feature.to_numpy()

pred_feature = pred_feature.reshape((1,20,-1))
# pred_feature.shape

import random as rn
import tensorflow as tf


for i in range(1, len(df)//1000+2):

        # 데이터 불러오기
        df_copy = df
     
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()
        model.add(GRU(16, 
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences = True,
                      activation='relu'))
        model.add(GRU(16, return_sequences=False, stateful=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        
    

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)



C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
34/34 [==============================] - ETA: 0s - loss: 0.1894
Epoch 1: val_loss improved from inf to 0.01656, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 32s 200ms/step - loss: 0.1894 - val_loss: 0.0166
Epoch 2/100
34/34 [==============================] - ETA: 0s - loss: 0.0232
Epoch 2: val_loss improved from 0.01656 to 0.00439, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 1s 30ms/step - loss: 0.0232 - val_loss: 0.0044
Epoch 3/100
32/34 [===========================>..] - ETA: 0s - loss: 0.0144
Epoch 3: val_loss did not improve from 0.00439
34/34 [==============================] - 1s 29ms/step - loss: 0.0141 - val_loss: 0.0044
Epoch 4/100
34/34 [==============================] - ETA: 0s - loss: 0.0120
Epoch 4: val_loss improved from 0.00439 to 0.00392, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 1s 28ms/step - loss: 0.0120 - val_loss: 0.0039
Epoch 5/100
34/34 [====

Epoch 23/100
69/69 [==============================] - ETA: 0s - loss: 0.0026
Epoch 23: val_loss improved from 0.00027 to 0.00023, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 2s 27ms/step - loss: 0.0026 - val_loss: 2.3263e-04
Epoch 24/100
69/69 [==============================] - ETA: 0s - loss: 0.0026
Epoch 24: val_loss did not improve from 0.00023
69/69 [==============================] - 2s 24ms/step - loss: 0.0026 - val_loss: 2.7555e-04
Epoch 25/100
69/69 [==============================] - ETA: 0s - loss: 0.0020
Epoch 25: val_loss did not improve from 0.00023
69/69 [==============================] - 2s 26ms/step - loss: 0.0020 - val_loss: 4.4593e-04
Epoch 26/100
68/69 [============================>.] - ETA: 0s - loss: 0.0019
Epoch 26: val_loss did not improve from 0.00023
69/69 [==============================] - 2s 25ms/step - loss: 0.0019 - val_loss: 4.2016e-04
Epoch 27/100
69/69 [==============================] - ETA: 0s - loss: 0.0020
Epoch 27: 

103/104 [============================>.] - ETA: 0s - loss: 0.0060
Epoch 13: val_loss improved from 0.00046 to 0.00040, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 3s 26ms/step - loss: 0.0060 - val_loss: 4.0275e-04
Epoch 14/100
102/104 [============================>.] - ETA: 0s - loss: 0.0066
Epoch 14: val_loss improved from 0.00040 to 0.00030, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 3s 27ms/step - loss: 0.0065 - val_loss: 2.9817e-04
Epoch 15/100
102/104 [============================>.] - ETA: 0s - loss: 0.0056
Epoch 15: val_loss did not improve from 0.00030
104/104 [==============================] - 3s 27ms/step - loss: 0.0056 - val_loss: 3.5921e-04
Epoch 16/100
104/104 [==============================] - ETA: 0s - loss: 0.0034
Epoch 23: val_loss did not improve from 0.00024
104/104 [==============================] - 4s 40ms/step - loss: 0.0034 - val_loss: 2.5682e-04
Epoch 24/100
103/104 [=================

104/104 [==============================] - ETA: 0s - loss: 8.6536e-04
Epoch 55: val_loss improved from 0.00018 to 0.00018, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 5s 48ms/step - loss: 8.6536e-04 - val_loss: 1.8035e-04
Epoch 56/100
103/104 [============================>.] - ETA: 0s - loss: 8.5238e-04
Epoch 56: val_loss did not improve from 0.00018
104/104 [==============================] - 4s 41ms/step - loss: 8.5780e-04 - val_loss: 2.2923e-04
Epoch 57/100
104/104 [==============================] - ETA: 0s - loss: 8.0856e-04
Epoch 57: val_loss improved from 0.00018 to 0.00018, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 5s 49ms/step - loss: 8.0856e-04 - val_loss: 1.7723e-04
Epoch 58/100
104/104 [==============================] - ETA: 0s - loss: 8.5053e-04
Epoch 58: val_loss did not improve from 0.00018
104/104 [==============================] - 5s 45ms/step - loss: 8.5053e-04 - val_loss: 2.4788e-04
Epoch 5

Epoch 8/100
137/139 [============================>.] - ETA: 0s - loss: 0.0049
Epoch 8: val_loss did not improve from 0.00038
139/139 [==============================] - 6s 40ms/step - loss: 0.0049 - val_loss: 4.1587e-04
Epoch 9/100
139/139 [==============================] - ETA: 0s - loss: 0.0040
Epoch 9: val_loss improved from 0.00038 to 0.00031, saving model to model\tmp_checkpoint.h5
139/139 [==============================] - 6s 46ms/step - loss: 0.0040 - val_loss: 3.1110e-04
Epoch 10/100
139/139 [==============================] - ETA: 0s - loss: 0.0035
Epoch 10: val_loss did not improve from 0.00031
139/139 [==============================] - 5s 39ms/step - loss: 0.0035 - val_loss: 3.5171e-04
Epoch 11/100
139/139 [==============================] - ETA: 0s - loss: 0.0031
Epoch 11: val_loss improved from 0.00031 to 0.00031, saving model to model\tmp_checkpoint.h5
139/139 [==============================] - 7s 52ms/step - loss: 0.0031 - val_loss: 3.0562e-04
Epoch 12/100
137/139 [========

Epoch 91/100
138/139 [============================>.] - ETA: 0s - loss: 0.0014
Epoch 91: val_loss did not improve from 0.00013
139/139 [==============================] - 5s 38ms/step - loss: 0.0014 - val_loss: 2.7247e-04
4000길이의 데이터 적용 완료
 길이: 4000, RMSE:6268.489558934109
[29300.527924892718, 13322.340750131056, 20433.23511565384, 6268.489558934109]
Epoch 1/100
174/174 [==============================] - ETA: 0s - loss: 0.0396
Epoch 1: val_loss improved from inf to 0.00281, saving model to model\tmp_checkpoint.h5
174/174 [==============================] - 50s 103ms/step - loss: 0.0396 - val_loss: 0.0028
Epoch 2/100
208/209 [============================>.] - ETA: 0s - loss: 0.0018
Epoch 3: val_loss did not improve from 0.00122
209/209 [==============================] - 14s 66ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 4/100
209/209 [==============================] - ETA: 0s - loss: 0.0016
Epoch 4: val_loss improved from 0.00122 to 0.00053, saving model to model\tmp_checkpoint.h5
209/

Epoch 7/100
244/244 [==============================] - ETA: 0s - loss: 2.2257e-04
Epoch 22: val_loss did not improve from 0.00052
244/244 [==============================] - 15s 62ms/step - loss: 2.2257e-04 - val_loss: 0.0015
Epoch 23/100
244/244 [==============================] - ETA: 0s - loss: 2.1146e-04
Epoch 23: val_loss improved from 0.00052 to 0.00047, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 45s 184ms/step - loss: 2.1146e-04 - val_loss: 4.6661e-04
Epoch 24/100
243/244 [============================>.] - ETA: 0s - loss: 2.0860e-04
Epoch 24: val_loss did not improve from 0.00047
244/244 [==============================] - 11s 45ms/step - loss: 2.0828e-04 - val_loss: 0.0021
Epoch 25/100
243/244 [============================>.] - ETA: 0s - loss: 2.1352e-04
Epoch 25: val_loss did not improve from 0.00047
244/244 [==============================] - 10s 41ms/step - loss: 2.1303e-04 - val_loss: 6.8355e-04
Epoch 26/100
242/244 [=====================

279/279 [==============================] - 12s 45ms/step - loss: 1.7259e-04 - val_loss: 0.0016
Epoch 27/100
277/279 [============================>.] - ETA: 0s - loss: 1.4611e-04
Epoch 27: val_loss did not improve from 0.00064
279/279 [==============================] - 8s 27ms/step - loss: 1.4581e-04 - val_loss: 0.0031
Epoch 28/100
277/279 [============================>.] - ETA: 0s - loss: 1.4114e-04
Epoch 28: val_loss did not improve from 0.00064
279/279 [==============================] - 7s 26ms/step - loss: 1.4296e-04 - val_loss: 0.0026
Epoch 29/100
277/279 [============================>.] - ETA: 0s - loss: 1.6016e-04
Epoch 29: val_loss did not improve from 0.00064
279/279 [==============================] - 7s 27ms/step - loss: 1.5985e-04 - val_loss: 8.3577e-04
Epoch 30/100
277/279 [============================>.] - ETA: 0s - loss: 1.4227e-04
Epoch 30: val_loss did not improve from 0.00064
279/279 [==============================] - 10s 35ms/step - loss: 1.4211e-04 - val_loss: 9.9278e

314/314 [==============================] - 8s 24ms/step - loss: 9.8662e-05 - val_loss: 2.9305e-04
Epoch 17/100
312/314 [============================>.] - ETA: 0s - loss: 1.0599e-04
Epoch 17: val_loss did not improve from 0.00026
314/314 [==============================] - 8s 25ms/step - loss: 1.0617e-04 - val_loss: 6.6600e-04
Epoch 18/100
313/314 [============================>.] - ETA: 0s - loss: 9.6866e-05
Epoch 18: val_loss did not improve from 0.00026
314/314 [==============================] - 8s 25ms/step - loss: 9.7359e-05 - val_loss: 2.9715e-04
Epoch 19/100
314/314 [==============================] - ETA: 0s - loss: 1.0046e-04
Epoch 19: val_loss did not improve from 0.00026
314/314 [==============================] - 7s 24ms/step - loss: 1.0046e-04 - val_loss: 2.8664e-04
Epoch 20/100
314/314 [==============================] - ETA: 0s - loss: 1.0559e-04
Epoch 20: val_loss did not improve from 0.00026
314/314 [==============================] - 9s 29ms/step - loss: 1.0559e-04 - val_los

In [3]:
print(predict)

[array([[170133.31]], dtype=float32), array([[188914.64]], dtype=float32), array([[171244.19]], dtype=float32), array([[188000.52]], dtype=float32), array([[198914.42]], dtype=float32), array([[190834.33]], dtype=float32), array([[184606.8]], dtype=float32), array([[191564.72]], dtype=float32), array([[205036.02]], dtype=float32), array([[187244.88]], dtype=float32)]
